<a href="https://colab.research.google.com/github/tsato-code/colab_notebooks/blob/main/hashibiro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Tue Apr 13 14:25:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
# 関連リポジトリをクローン
%cd /content
!git clone https://github.com/KaiyangZhou/vsumm-reinforce

/content
Cloning into 'vsumm-reinforce'...
remote: Enumerating objects: 121, done.
remote: Total 121 (delta 0), reused 0 (delta 0), pack-reused 121
Receiving objects: 100% (121/121), 641.05 KiB | 16.87 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [3]:
# データセットと学習済みモデルをダウンロード
%cd vsumm-reinforce
#!wget http://www.eecs.qmul.ac.uk/~kz303/vsumm-reinforce/datasets.tar.gz
#!tar -xvzf datasets.tar.gz
#!wget http://www.eecs.qmul.ac.uk/~kz303/vsumm-reinforce/models.tar.gz
#!tar -xvzf models.tar.gz

/content/vsumm-reinforce


In [4]:
%cd ..
# GoogLeNet for Image Classification
!git clone https://github.com/conan7882/GoogLeNet-Inception-tf
%cd GoogLeNet-Inception-tf
!wget https://www.dropbox.com/sh/axnbpd1oe92aoyd/AADJaXakFvqOH8sXkdu6guHta/googlenet.npy
%cd ..
# Kernel Temporal Segmentation
!wget http://pascal.inrialpes.fr/data2/potapov/med_summaries/kts_ver1.1.tar.gz
!tar -zxvf kts_ver1.1.tar.gz

/content
Cloning into 'GoogLeNet-Inception-tf'...
remote: Enumerating objects: 332, done.
remote: Total 332 (delta 0), reused 0 (delta 0), pack-reused 332
Receiving objects: 100% (332/332), 11.78 MiB | 20.55 MiB/s, done.
Resolving deltas: 100% (123/123), done.
/content/GoogLeNet-Inception-tf
--2021-04-13 14:25:34--  https://www.dropbox.com/sh/axnbpd1oe92aoyd/AADJaXakFvqOH8sXkdu6guHta/googlenet.npy
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/axnbpd1oe92aoyd/AADJaXakFvqOH8sXkdu6guHta/googlenet.npy [following]
--2021-04-13 14:25:35--  https://www.dropbox.com/sh/raw/axnbpd1oe92aoyd/AADJaXakFvqOH8sXkdu6guHta/googlenet.npy
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc97dc01e822b85572caf0a3a90c.dl.dropboxusercontent.

In [5]:
!mkdir -p data/original
%cd data/original

/content/data/original


In [6]:
!pip install pytube
from pytube import YouTube

data_dir = '/content/data/original/'
# hot pepper beauty
YouTube("https://www.youtube.com/watch?v=rwt3oS_mFVQ").streams.first().download(data_dir+'test01')
# GREEN DAKARA
YouTube("https://www.youtube.com/watch?v=cShbQ2pQs94").streams.first().download(data_dir+'test02')
# ポカリスウェット
YouTube("https://www.youtube.com/watch?v=cKM2HQLK8Pg").streams.first().download(data_dir+'test03')

'/content/data/original/test03/ポカリスエットCM｜「でも君が見えた」篇 30秒 B.mp4'

In [7]:
# mp4ファイルを移動
!mv /content/data/original/*/*.mp4 /content/data/original

In [51]:
# スクリプトを修正
text = r'''
#!/bin/bash

"""
This script decompose a video into frames
How to use: replace path_to_videos and path_to_frames with real paths
"""

for f in /content/data/original/*.mp4
do
  echo "Processing $f file..."
  # take action on each file. $f store current file name\
  basename=$(ff=${f%.ext} ; echo ${ff##*/})
  name=$(echo $basename | cut -d'.' --complement -f2-)
  echo $f
 mkdir -p /content/data/frames/"$name"
 ffmpeg -i "$f" -f image2 -qscale:v 2 /content/data/frames/"$name"/%06d.jpg
done
'''
with open('/content/vsumm-reinforce/extra-tools/videos2frames.sh', 'w') as f:
    f.write(text)

In [9]:
# 修正したスクリプトを実行
%cd /content/vsumm-reinforce/extra-tools
!./videos2frames.sh

/content/vsumm-reinforce/extra-tools
./videos2frames.sh: line 6: $'\nThis script decompose a video into frames\nHow to use: replace path_to_videos and path_to_frames with real paths\n': command not found
Processing /content/data/original/ポカリスエットCM｜「でも君が見えた」篇 30秒 B.mp4 file...
/content/data/original/ポカリスエットCM｜「でも君が見えた」篇 30秒 B.mp4
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enab

In [25]:
!pip install tensorflow==1.13.1

     |████████████████████████████████| 92.6MB 60kB/s 
     |████████████████████████████████| 368kB 14.3MB/s 
     |████████████████████████████████| 3.2MB 41.9MB/s 
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0


In [52]:
# 特徴量抽出のコード
text = r'''
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# File: inception_pretrained.py
# Author: Qian Ge <geqian1001@gmail.com>

import os
import sys
import platform
import argparse
import numpy as np
import tensorflow as tf

sys.path.append('../')
import loader as loader
from src.nets.googlenet import GoogLeNet
import h5py
import src.models.layers as L

# PRETRINED_PATH = '/home/qge2/workspace/data/pretrain/inception/googlenet.npy'
# DATA_PATH = '../data/'
PRETRINED_PATH = '/content/GoogLeNet-Inception-tf/googlenet.npy'
DATA_PATH = '/content/data/frames/test01'
DATASET_PATH = '/content/vsumm-reinforce/datasets/eccv16_dataset_pokari_google_pool5.h5'
IM_CHANNEL = 3


def get_args():
    parser = argparse.ArgumentParser()

    parser.add_argument('--pretrained_path', type=str, default=PRETRINED_PATH,
                        help='Directory of pretrain model')
    parser.add_argument('--im_name', type=str, default='.jpg',
                        help='Part of image name')
    parser.add_argument('--data_path', type=str, default=DATA_PATH,
                        help='Directory of test images')
    
    return parser.parse_args()

def test_pre_trained():
    FLAGS = get_args()
    # Read ImageNet label into a dictionary
    label_dict = loader.load_label_dict()
    # Create a Dataflow object for test images
    image_data = loader.read_image(
        im_name=FLAGS.im_name, n_channel=IM_CHANNEL,
        data_dir=FLAGS.data_path, batch_size=1)

    # Create a testing GoogLeNet model
    test_model = GoogLeNet(
        n_channel=IM_CHANNEL, n_class=1000, pre_trained_path=FLAGS.pretrained_path)
    test_model.create_test_model()

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        dataset = h5py.File(DATASET_PATH, 'w')
        video_name = FLAGS.data_path.split('/')[-2]
        features = np.empty((1, 1024), dtype=np.float64)  # Put dummy

        while image_data.epochs_completed < 1:
            # read batch files
            batch_data = image_data.next_batch_dict()
            # get batch file names
            batch_file_name = image_data.get_batch_file_name()[0]
            # get prediction results
            #pred = sess.run(test_model.layers['top_5'],
            feature = sess.run(L.global_avg_pool(test_model.layers['inception_out']),
                            feed_dict={test_model.image: batch_data['image']})
            features = np.concatenate((features, feature))

        dataset.create_dataset('{}/features_origin'.format(video_name), data=features[1:])
        dataset.close()

if __name__ == "__main__":
    test_pre_trained()
'''
with open('/content/GoogLeNet-Inception-tf/examples/feature_extraction.py', 'w') as f:
    f.write(text)

In [49]:
%cd /content/GoogLeNet-Inception-tf/examples

# これは一時的に必要なディレクトリ
!mkdir -p /content/vsumm-reinforce/datasets
# ディレクトリのパス設定次第でok
!python feature_extraction.py

/content/GoogLeNet-Inception-tf/examples
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:

In [34]:
text = r'''
import numpy as np
from cpd_auto import cpd_auto
import h5py

DATASET_PATH = "../vsumm-reinforce/datasets/eccv16_dataset_pokari_google_pool5.h5"
SAMP_RATE = 15  # sampling rate (fps)
N_CHANGE_POINTS = 30  # the number of change points

if __name__ == "__main__":
    dataset = h5py.File(DATASET_PATH, "r+")

    for video_name in dataset.keys():
        X = dataset[video_name]["features_origin"][...]
        n = X.shape[0]
        K = np.dot(X, X.T)
        cps, scores = cpd_auto(K, N_CHANGE_POINTS, 1)
        print("Estimated: (m=%d)" % len(cps), cps)

        cps = np.concatenate(([0], cps))
        cps = np.concatenate((cps, [n]))

        # Update dataset
        features = np.array([frame for i, frame in enumerate(X) if i % SAMP_RATE == 0], dtype=np.float64)
        change_points = np.array([[cps[i], cps[i + 1] - 1] for i in range(cps.shape[0] - 1)], dtype=np.int64)
        n_frames = np.array(n, dtype=np.int64)
        n_frame_per_seg = np.array([(p[1] - p[0] + 1) for p in change_points], dtype=np.int64)
        picks = np.array([(i * SAMP_RATE) for i in range((n - 1) // SAMP_RATE + 1)], dtype=np.int64)

        key = dataset[video_name]
        dataset.create_dataset(key + "/features", data=features)
        dataset.create_dataset(key + "/change_points", data=change_points)
        dataset.create_dataset(key + "/n_frames", data=n_frames)
        dataset.create_dataset(key + "/n_frame_per_seg", data=n_frame_per_seg)
        dataset.create_dataset(key + "/picks", data=picks)

    dataset.close()
'''
with open('/content/kts_ver1.1/segment.py', 'w') as f:
    f.write(text)

In [38]:
text = r'''
import numpy as np


def calc_scatters(K):
    """Calculate scatter matrix: scatters[i,j] = {scatter of the sequence with
    starting frame i and ending frame j}
    """
    n = K.shape[0]
    K1 = np.cumsum([0] + list(np.diag(K)))
    K2 = np.zeros((n + 1, n + 1))
    # TODO: use the fact that K - symmetric
    K2[1:, 1:] = np.cumsum(np.cumsum(K, 0), 1)

    diagK2 = np.diag(K2)

    i = np.arange(n).reshape((-1, 1))
    j = np.arange(n).reshape((1, -1))
    scatters = (
            K1[1:].reshape((1, -1)) - K1[:-1].reshape((-1, 1)) -
            (diagK2[1:].reshape((1, -1)) + diagK2[:-1].reshape((-1, 1)) -
             K2[1:, :-1].T - K2[:-1, 1:]) /
            ((j - i + 1).astype(np.float32) + (j == i - 1).astype(np.float32))
    )
    scatters[j < i] = 0

    return scatters


def cpd_nonlin(K, ncp, lmin=1, lmax=100000, backtrack=True, verbose=True,
               out_scatters=None):
    """Change point detection with dynamic programming
    :param K: Square kernel matrix
    :param ncp: Number of change points to detect (ncp >= 0)
    :param lmin: Minimal length of a segment
    :param lmax: Maximal length of a segment
    :param backtrack: If False - only evaluate objective scores (to save memory)
    :param verbose: If true, print verbose message
    :param out_scatters: Output scatters
    :return: Tuple (cps, obj_vals)
        - cps - detected array of change points: mean is thought to be constant
            on [ cps[i], cps[i+1] )
        - obj_vals - values of the objective function for 0..m changepoints
    """
    m = int(ncp)  # prevent numpy.int64

    n, n1 = K.shape
    assert n == n1, 'Kernel matrix awaited.'
    assert (m + 1) * lmin <= n <= (m + 1) * lmax
    assert 1 <= lmin <= lmax

    if verbose:
        print('Precomputing scatters...')
    J = calc_scatters(K)

    if out_scatters is not None:
        out_scatters[0] = J

    if verbose:
        print('Inferring best change points...')
    # I[k, l] - value of the objective for k change-points and l first frames
    I = 1e101 * np.ones((m + 1, n + 1))
    I[0, lmin:lmax] = J[0, lmin - 1:lmax - 1]

    if backtrack:
        # p[k, l] --- 'previous change' --- best t[k] when t[k+1] equals l
        p = np.zeros((m + 1, n + 1), dtype=int)
    else:
        p = np.zeros((1, 1), dtype=int)

    for k in range(1, m + 1):
        for l in range((k + 1) * lmin, n + 1):
            tmin = max(k * lmin, l - lmax)
            tmax = l - lmin + 1
            c = J[tmin:tmax, l - 1].reshape(-1) + \
                I[k - 1, tmin:tmax].reshape(-1)
            I[k, l] = np.min(c)
            if backtrack:
                p[k, l] = np.argmin(c) + tmin

    # Collect change points
    cps = np.zeros(m, dtype=int)

    if backtrack:
        cur = n
        for k in range(m, 0, -1):
            cps[k - 1] = p[k, cur]
            cur = cps[k - 1]

    scores = I[:, n].copy()
    scores[scores > 1e99] = np.inf
    return cps, scores
'''
with open('/content/kts_ver1.1/cpd_nonlin.py', 'w') as f:
    f.write(text)

In [48]:
%cd /content/kts_ver1.1
# python2との違いでエラーになってるっぽい。
# もしかしたらh5ファイルを読み込むところから書き換えが必要？
!python segment.py

/content/kts_ver1.1
Precomputing scatters...
Inferring best change points...
Precomputing scatters...
Inferring best change points...
Estimated: (m=30) [ 43  70 100 144 158 168 179 186 217 237 270 294 302 332 362 409 451 506
 514 548 562 585 612 633 662 683 704 730 751 778]
****************************************
frames
<HDF5 group "/frames" (1 members)>
<class 'h5py._hl.group.Group'>
<KeysViewHDF5 ['features_origin']>
['_MutableMapping__marker', '__abstractmethods__', '__bool__', '__class__', '__contains__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__setitem__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_i